# Project 1: Customer Database
**This is the first of three mandatory projects to be handed in as part of the assessment for the course 02807 Computational Tools for Data Science at Technical University of Denmark, autumn 2019.**

#### Practical info
- **The project is to be done in groups of at most 3 students**
- **Each group has to hand in _one_ Jupyter notebook (this notebook) with their solution**
- **The hand-in of the notebook is due 2019-10-13, 23:59 on DTU Inside**

#### Your solution
- **Your solution should be in Python**
- **For each question you may use as many cells for your solution as you like**
- **You should document your solution and explain the choices you've made (for example by using multiple cells and use Markdown to assist the reader of the notebook)**
- **You should not remove the problem statements, and you should not modify the structure of the notebook**
- **Your notebook should be runnable, i.e., clicking [>>] in Jupyter should generate the result that you want to be assessed**
- **You are not expected to use machine learning to solve any of the exercises**
- **You will be assessed according to correctness and readability of your code, choice of solution, choice of tools and libraries, and documentation of your solution**

## Introduction
Your team has been hired by the company X as data scientists. X makes gadgets for a wide range of industrial and commercial clients.

As in-house data scientists, your teams first task, as per request from your new boss, is to optimize business operations. You have decided that a good first step would be to analyze the companys historical sales data to gain a better understanding of where profit is coming from. It may also reveal some low hanging fruit in terms of business opportunities.

To get started, you have called the IT department to get access to the customer and sales transactions database. To your horror you've been told that such a database doens't exist, and the only record of sales transactions is kept by John from finance in an Excel spreadsheet. So you've emailed John asking for a CSV dump of the spreadsheet...

In this project you need to clean the data you got from John, enrich it with further data, prepare a database for the data, and do some data analysis. The project is comprised of five parts. They are intended to be solved in the order they appear, but it is highly recommended that you read through all of them and devise an overall strategy before you start implementing anything.

## Part 1: Cleaning the data
John has emailed you the following link to the CSV dump you requested.

- [transactions.csv](https://raw.githubusercontent.com/patrickcording/02807-comp-tools/master/docker/work/data/transactions.csv)

It seems as though he has been a bit sloppy when keeping the records. 

In this part you should:
- Explain what the data is
- Clean it to prepare it for inserting into a database and doing data analysis 

---
### Solution

In [58]:
# Importing the necessary libraries.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv("transactions.csv", encoding='utf-8-sig')


#The following is a feature description of the 6 features in the dataset

#Part:
#This seems to be a partnumber (string). To investigate the purity and nature of the data 
#the part number was analysed using command line. The following command revealed best results:
#cut -d , -f 1 transactions.csv | sort | uniq -c
#It showed that majority of part numbers were used around 200 times
#10 records countained an empty part number
#the "real part numbers" had format: 
#4 or 5 digits followed by '-' followed by 3 or 4 digits
#In total there are 100 different part numbers (excluding the empty part number)
i=0
for index, row in df.iterrows():
    if (row['company'] == 'Avaveo' and row['country']!='France'):
        row['country'] = 'France'
    
    elif (row['company'] == 'Brainsphere'):
        if(row['country']!= 'Portugal'):
            row['country'] = 'Portugal'
        if(row['city']!='Braga'):
            row['city'] = 'Braga'
            
    elif(row['company']== 'Bubblemix' and row['country'] !='Japan'):
        row['country'] = 'Japan'
        
    elif(row['company']=='Buzzbean' and row['country'] != 'Germany'):
        row['country'] = 'Germany'
        
    elif(row['company']=='Chatterbridge' and row['country']!= 'Spain'):
        row['country']='Spain'
        
    elif(row['company']=='Eimbee' and row['country']!= 'France'):
        row['country']='France'

    elif(row['company']=='Flipstorm'):
        if(row['city'] == 'Athens' and row['country'] != 'Greece'):
            row['country']='Greece'
        if(row['city'] == 'Nanterre' and row['country'] != 'France'):
            row['country']='France'
    
    elif(row['company']=='Gabcube' and row['country']!= 'Portugal'):
        row['country']='Portugal'    
    
    elif(row['company']=='Gabtune' and row['country']!= 'France'):
        row['country']='France'

    elif(row['company']=='Gevee' and row['country']!= 'France'):
        row['country']='France'
        
    elif(row['company']=='Innojam' and row['country']!= 'Netherlands'):
        row['country']='Netherlands'
        
    elif(row['company']=='Kanoodle'):
        if(row['country']!= 'Netherlands'):
            row['country']='Netherlands'
        if(row['city']!='Niihama'):
            row['city']='Niihama'
     
    elif(row['company'] == 'Laj0'):
        row['company'] = 'Lajo'
        
    elif(row['company']=='Lajo' and row['country']!= 'Greece'):
        row['country']='Greece'
        
    elif(row['company'] == 'Ntagz'):
        row['company'] = 'Ntags'
        
    elif(row['company']=='Ntags'):
        if(row['country']!= 'Portugal'):
            row['country']='Portugal'
        if(row['city']!='Lisbon'):
            row['city']='Lisbon'
            
    elif(row['company']=='Realpoint' and row['country']!= 'Portugal'):
        row['country']='Portugal'
        
    elif(row['company']=='Thycero' and row['country']!= 'France'):
        row['country']='France'
        
    elif(row['company']=='Riffpath' and row['country']!= 'Greece'):
        row['country']='Greece'
        
    elif(row['company']=='Roodel' and row['country']!= 'Portugal'):
        row['country']='Portugal'
        
    elif(row['company']=='Shufflebeat'):
        if(row['country']!= 'Portugal'):
            row['country']='Portugal'
        if(row['city']!='Porto'):
            row['city']='Porto'
            
    elif(row['company']=='Tagtune' and row['country']!= 'Switzerland'):
        row['country']='Switzerland'
        
    elif(row['company']=='Teklist'):
        if(row['country']!= 'Netherlands'):
            row['country']='Netherlands'
        if(row['city']!='Arnhem'):
            row['city']='Arnhem'
            
    elif(row['company'] == 'Thoughtmixz'):
        row['company'] = 'Thoughtmix'
        
    elif(row['company']=='Thoughtmix'):
        if(row['country']!= 'Portugal'):
            row['country']='Portugal'
        if(row['city']!='Amadora'):
            row['city']='Amadora'
            
    elif(row['company']=='Twitterbeat'):
        if(row['country']!= 'France'):
            row['country']='France'
        if(row['city']!='Annecy'):
            row['city']='Annecy'
            
    elif(row['company']=='Voomm' and row['country']!= 'France'):
        row['country']='France'
        
    elif(row['company']=='Wordify'):
        if(row['country']!= 'United States'):
            row['country']='United States'
        if(row['city']!='New York'):
            row['city']='New York'
            
    elif(row['company']=='Yozio'):
        if(row['country']!= 'Greece'):
            row['country']='Greece'
        if(row['city']!='Patras'):
            row['city']='Patras'
            
    elif(row['company']=='Zoonder'):
        if(row['country']!= 'United States'):
            row['country']= 'United States'
        if(row['city']!='Boston'):
            row['city']='Boston'
            
    elif(row['company'] == 'Zooxo.'):
        row['company'] = 'Zooxo'        
            
    elif(row['company']=='Zooxo'):
        if(row['country']!= 'United Kingdom'):
            row['country']= 'United Kingdom'
        if(row['city']!='London'):
            row['city']='London'
            
    elif(row['company'] == ' -'):
        row['company'] = 'Zoonder'
        
    elif(row['company'] == ' a'):
        row['company'] = 'Wordify'
        
    elif(row['company'] == 'aa'):
        row['company'] = 'Wordify'
            

            
#for index, row in df.iterrows():
#    if (row['company'] == 'Zoonder' and row['country']=='United States'):
#        i+=1
#print(i)
        
        
#Company
#This seems to be a company name (string)
#To analyse the purity/nature following command was used:
#cut -d , -f 2 transactions.csv | sort | uniq -c
#36 different companies exist
#Most appearing company is 'Thoughtmix' which appears 2795 times (raw data)
#However 'Thoughtmix' also appears as 'Thoughtmixz' (this is assumed to be spelling error)
#Apart from spelling errors there seem to be 3 outliers:
#'-', 'a', 'aa' which all appear a single time

#Country
#This seems to be the country, where the company is placed (string)
#To analyse the purity/nature following command was used:
#cut -d , -f 3 transactions.csv | sort | uniq -c
#15 different countries were listed
#The 'empty' country appeared 2171 times so it seems like John forgot to not the real company multiple times
#Some as for company names some country names are listed wrongly:
#e.g. 'US' is used once in stead of 'United States'
#e.g. 'Tyskland' is used once in stead of 'Germany'

#City 
#This seems to be the city, where the company is placed (string)
#To analyse the purity/nature following command was used:
#cut -d , -f 4 transactions.csv | sort | uniq -c
# This gives 32 different cities
#The whitespace sicty appear 33 times
#Seems like there are few/no spelling mistakes
#However, some values appear only once, so maybe the city was typed wrongly
#for some companied/contries

#To fix issues for column: 'company', 'country', 'city' (including many empty countries), following command is useful:
#cut -d , -f 2-4 transactions.csv | sort | uniq -c






#Price

#negative values, e.g.  $-153.91
#values with $ first, e.g. $1.0
#empty value 

#cut -d , -f 5 transactions.csv | sort | uniq -c| head


#inspecting what they can start with: 
#cut -d , -f 5 transactions.csv | cut -c 1-1 | sort | uniq -c 
#Normal values: $, 1-9, Pund, Yen?
#Odd values: empty: 1, '-' (negative values): 61, 'n': 3, 'v': 2

#cut -d , -f 5 transactions.csv | grep n
#shows three entries have value 'na'

#cut -d , -f 5 transactions.csv | grep v
#shows two entries have value 'void'

#cut -d , -f 5 transactions.csv | grep - | head
#e.g.: '-181.47€', '£-91.3'

#using cmd: cut -d , -f 5 transactions.csv | grep € | cut -c 1-1 | sort | uniq
#we can see that lines with € generally end with € and doesn't have other monetary value at start
#Thus, since this is only monetary value which seems to be at the end, it seems fair to assume
#That no line contains two different monetary declarations (e.g. $12.12€)

#cut -d , -f 5 transactions.csv | grep € | grep -
#shows one line has value '-'

#=> empty value, '-' value, 'n' values, 'v' values.
#=> negative values treated as positive values
#=> positive values: extract monetary value from first or last char

#in python script do something like
#if starts with $ - dollar
#else if starts with € - euro
#else: Alert(has no monetary value) - to check if any lines have zero monetary declaration
#also check for double monetary declaration? 

#Date
#This seems to be the date for the transaction, consisting of a date and a specific time
#

#cut -d , -f 6 transactions.csv | cut -d ' ' -f 1 | sort | uniq -c | head
#7 10/04/2017
#19 2016-01-02
# same query with tail only reveals regular date formatted values:
#14 2018-12-31
#1 2019-02-21
#1 2019-05-14
#1 date

#Inspect time parameter: 
#cut -d , -f 6 transactions.csv | cut -d ' ' -f 2 | sort | uniq -c | head
#9 
#2 00:00:05
#2 00:00:06
#Same query with tail shows regular formatting (no value larger than 23:59:59)











455


In [59]:
df.astype({'company':'str'})
df.dtypes

part       object
company    object
country    object
city       object
price      object
date       object
dtype: object

---
## Part 2: Enriching the data

A common task for a data scientists is to combine or enrich data from internal sources with data available from external sources. The purpose of this can be either to fix issues with the data or to make it easier to derive insights from the data.

In this part you should enrich your data with data from at least one external source. You may look to part 4 for some  inspiration as to what is required. Your solution should be automated, i.e., you can not ask the reader of your notebook to download any data manually. You should argue why and what you expect to achieve by the enrichments you are doing.

---
### Solution

---
## Part 3: Creating a database
Storing data in a relational database has the advantages that it is persistent, fast to query, and it will be easier access for other employees at Weyland-Yutani.

In this part you should:
- Create a database and table(s) for the data
- Insert data into the tables

You may use SQLite locally to do this. You should argue why you choose to store your data the way you do. 

---
### Solution

In [9]:
import sqlite3
#Connect to the database and create tables
conn = sqlite3.connect('weylandYutani.sqlite')
c = conn.cursor()
# Create Table 1: Company (primary key (id)[int], company name[string], 
#                    company country[string], company city[string])
c.execute('CREATE TABLE IF NOT EXISTS companies(company_id PRIMARY KEY AUTOINCREMENT, name TEXT, country TEXT, city TEXT)')
# Create Table 2: (transaction id (primary key)[int], part(id)[int], 
#           price[int], date[date], time[time], companyid[int])
c.execute('CREATE TABLE IF NOT EXISTS transactions(transaction_id INTEGER PRIMARY KEY AUTOINCREMENT, part_id INTEGER, price INTEGER, dateTime TEXT, company_id INTEGER)')
conn.commit()


## Import data into the table


# Maybe sort the list with unique values. And put it into a list. Then fill it into the database directly
# for each transaction in the transactions. Look at the company name. If it contains data 
#    and there does not already exists a company with that name - Add it to the table. 
#     If it already exists - break
listOfCompanies = [1,2,3]
for company in listOfCompanies:
    name = company.name
    country = company.country
    city = company.city
    c.execute(f"INSERT INTO companies(name, country, city) VALUES ({name},{country},{city})")

    
# for each transaction insert it into the DB
for transaction in transactions:
    part_id = transaction.part_id
    price = transaction.price
    dateTime = transaction.dateTime
    # Check what company it is and extract the id. 
    company_name = str(transaction.company_name)
    c.execute(f"SELECT company_id FROM companies WHERE name == {company_name}")
    company_id = c.fetchone()
    c.execute(f"INSERT INTO transactions (part_id, price, dateTime, company_id) VALUES ({part_id},{price},{dateTime},{company_id})")


---
## Part 4: Analyzing the data
You are now ready to analyze the data. Your goal is to gain some actionable business insights to present to your boss. 

In this part, you should ask some questions and try to answer them based on the data. You should write SQL queries to retrieve the data. For each question, you should state why it is relevant and what you expect to find.

To get you started, you should prepare answers to the following questions. You should add more questions.
#### Who are the most profitable clients?
Knowing which clients that generate the most revenue for the company will assist your boss in distributing customer service ressources.

#### Are there any clients for which profit is declining?
Declining profit from a specific client may indicate that the client is disatisfied with the product. Gaining a new client is often much more work than retaining one. Early warnings about declining profit may help your boss fighting customer churn.


Remember, you are taking this to your new boss, so think about how you present the data.

---
### Solution

---
## Part 5: Performance
Your boss is very impressed with what you have achieved in less than two weeks, and he would like to take your idea of storing the customer and sales data in a relational database to production. However, John is concerned that the solution will not scale. His experience is telling him that you will see many occurrences of the following queries.

- Show all sales to company X between time $t_1$ and time $t_2$
- Show the latest X sales in the database
- Show total sales per company per day

Show that Johns concern is not justified.

---
### Solution

---